In [ ]:
# 1번
#TODO: 각 패널 엣지에 x, y 에 대한 좌표 정보를 얻기 위한 함수
import pandas as pd

def process_path_data(data):
    """
    주어진 Path 데이터를 DataFrame으로 처리.
    - 요소 타입 변환 (QuadraticBezier -> Bezier Curve, Line -> Straight)
    - start, control, control1, control2, end, radius의 실수부/허수부를 x, y로 분리
    - 복소수 열 제거
    
    Args
        data (tuple): Path 객체와 속성 정보가 포함된 데이터
    
    Returns:
        pd.DataFrame: 처리된 데이터프레임
    """
    rows = []
    # Path 내부 요소 순회
    for element in data[0]:
        row = {
            "type": type(element).__name__,  # 요소 타입
            "start": getattr(element, "start", None),
            "controla": getattr(element, "control", None),
            "controlb": getattr(element, "control", None),
            "control1": getattr(element, "control1", None),
            "control2": getattr(element, "control2", None),
            "end": getattr(element, "end", None),
            "radius": getattr(element, "radius", None),
            "rotation": getattr(element, "rotation", None),
            "large_arc": getattr(element, "large_arc", None),
            "sweep": getattr(element, "sweep", None),
        }
        rows.append(row)
    
    # DataFrame 생성
    df = pd.DataFrame(rows)
    
    # type 열의 값을 변경하기 위한 매핑 사전
    type_mapping = {
        "QuadraticBezier": "Bezier Curve",
        "CubicBezier": "Bezier Curve",
        "Line": "Straight",
        "Arc": "Arc", #임의로 바꿈    
    }
    
    # type 열 값 변경
    df["type"] = df["type"].map(type_mapping).fillna(df["type"])
    
    # TODO: 2024.01.16 이후 Arc에 대한 정보에 대해 Bezier Curve로 변환예정.
    # -> 일단 되게 끔 하는게 목표.
    # 대상 열 목록
    coordinate_columns = ["start",  "controla", "controlb", "control1", "control2",  "end"] 
    
    # 실수부와 허수부를 분리하여 새로운 열로 추가하는 함수
    def extract_coordinates(df, columns):
        for col in columns:
            df[f"{col}_x"] = df[col].apply(lambda z: z.real if isinstance(z, complex) else None)
            df[f"{col}_y"] = df[col].apply(lambda z: z.imag if isinstance(z, complex) else None)
        # TODO: x, y 좌표를 상하 반전
        # minimize
        max_y = df[['start_y', 'controla_y', 'controlb_y', 'control1_y', 'control2_y', 'end_y']].max().max()
        min_y = df[['start_y', 'controla_y', 'controlb_y', 'control1_y', 'control2_y', 'end_y']].min().min()
        min_x = df[['start_x', 'controla_x', 'controlb_x', 'control1_x', 'control2_x', 'end_x']].min().min()
        max_x = df[['start_x', 'controla_x', 'controlb_x', 'control1_x', 'control2_x', 'end_x']].max().max()
        
        stand_y = (max_y + min_y)
        stand_x = (max_x + min_x)
        for col in columns: 
            df[f"{col}_y"] = (stand_y - df[f"{col}_y"]) * 10 #TODO: 01.17 scaling까지 
            df[f"{col}_x"] = df[f"{col}_x"]* 10 #TODO: scaling까지 
        df['controlb_x'] = df['controla_x'].round(4) # TODO: ontrolb_x 에 round
        
        # TODO: 여기까지
        return df
    
    # 실수부와 허수부를 분리하여 데이터프레임에 추가
    df = extract_coordinates(df, coordinate_columns)
    
    # 기존 열 중 복소수 값을 제거
    df = df.drop(columns=coordinate_columns)
    
    return df

In [1]:
# TODO:아래에 해당하는 arc_to_bezier는 개구려서 새로 만든거임
import numpy as np
import re

def arc_to_single_control_bezier(arc):
    """
    Arc 데이터를 단일 Control Point를 가진 Bezier Curve로 변환합니다.
    
    Args:
        arc (dict): Arc 데이터 (start_x, start_y, end_x, end_y, radius, large_arc, sweep 포함).
    
    Returns:
        dict: Bezier Curve의 시작점, 끝점, 제어점 1개.
    """
    # 시작점과 끝점 정의
    start_x, start_y = arc["start_x"], arc["start_y"]
    end_x, end_y = arc["end_x"], arc["end_y"]
    radius = arc["radius"]  # 반지름
    large_arc = arc["large_arc"]
    sweep = arc["sweep"]

    # 시작점과 끝점을 복소수로 변환
    start = np.array([start_x, start_y])
    end = np.array([end_x, end_y])

    # 중심 계산
    center = (start + end) / 2
    cx, cy = center[0], center[1]

    # 각도 계산
    theta_start = np.arctan2(start_y - cy, start_x - cx)
    theta_end = np.arctan2(end_y - cy, end_x - cx)

    # sweep 및 large_arc 처리
    if not sweep:
        theta_start, theta_end = theta_end, theta_start
    if large_arc and abs(theta_end - theta_start) < np.pi:
        theta_end += 2 * np.pi

    # 중간 각도 계산
    theta_m = (theta_start + theta_end) / 2  # 중간 각도

    # Control Point 계산 (중간 각도 기준)
    control_x = cx + radius * np.cos(theta_m)
    control_y = cy + radius * np.sin(theta_m)
    control_point = np.array([control_x, control_y])

    # Control Point를 직선(start-end) 기준으로 대칭 변환
    def reflect_point_over_line(p, p0, p1):
        # 직선 벡터
        line_vec = p1 - p0
        line_vec /= np.linalg.norm(line_vec)  # 정규화
        # 점과 직선의 수직 대칭 계산
        proj = np.dot(p - p0, line_vec) * line_vec
        reflection = 2 * proj - (p - p0)
        return p0 + reflection

    # Control Point 대칭
    control_reflected = reflect_point_over_line(control_point, start, end)

    # 반환 형식
    return {
        "start_x": start_x, "start_y": start_y,
        "control1_x": control_reflected[0], "control1_y": control_reflected[1],
        "control2_x": control_reflected[0].round(4), "control2_y": control_reflected[1],
        "end_x": end_x, "end_y": end_y,
    }

def select_direction_front(fstart_fend):
    edge_str= fstart_fend["edge"]
    pattern = r'start=\(([^)]+)\).*end=\(([^)]+)\)'
    match = re.search(pattern, edge_str)
    if match:
        start_str = match.group(1)  # "8.92133247524254+33.21647810564835j"
        end_str   = match.group(2)  # "0.7116916666666668+34.66147782685373j"
        
        # 문자열 -> 복소수 변환
        start_complex = complex(start_str)  # (8.92133247524254+33.21647810564835j)
        end_complex   = complex(end_str)    # (0.7116916666666668+34.66147782685373j)

        # 실수부, 허수부 분리
        x1 = start_complex.real # start_x
        y1 = start_complex.imag # start_y
        x2   = end_complex.real # end_x
        y2   = end_complex.imag # end_y

    if (x1 < x2 and y1 < y2) or (x1 > x2 and y1 >= y2) or (x1 == x2 and y1  > y2) :
        return True
    elif (x1 <x2 and y1 >= y2) or (x1 > x2 and y1 < y2) or (x1 == x2 and y1 < y2) :
        return False

      

def select_direction_back(fstart_fend):
    edge_str= fstart_fend["edge"]
    pattern = r'start=\(([^)]+)\).*end=\(([^)]+)\)'
    match = re.search(pattern, edge_str)
    if match:
        start_str = match.group(1)  # "8.92133247524254+33.21647810564835j"
        end_str   = match.group(2)  # "0.7116916666666668+34.66147782685373j"
        
        # 문자열 -> 복소수 변환
        start_complex = complex(start_str)  # (8.92133247524254+33.21647810564835j)
        end_complex   = complex(end_str)    # (0.7116916666666668+34.66147782685373j)

        # 실수부, 허수부 분리
        x1 = start_complex.real # start_x
        y1 = start_complex.imag # start_y
        x2   = end_complex.real # end_x
        y2   = end_complex.imag # end_y

    # back panel stitch edge dirceciton decision 인 경우
    if (x1 < x2 and y1 < y2) or (x1 > x2 and y1 >= y2) or (x1 == x2 and y1  > y2) :
        return False
    elif (x1 <x2 and y1 >= y2) or (x1 > x2 and y1 < y2) or (x1 == x2 and y1 < y2) :
        return True     
# 테스트 데이터
# arc_data = {
#     "start_x": 130.1925,
#     "start_y": 398.36598,
#     "end_x": 238.3805,
#     "end_y": 298.021843,
#     "radius": 108.49457592383715,  # 반지름
#     "large_arc": False,
#     "sweep": True,
# }

# # 변환 실행
# bezier_curve_data = arc_to_single_control_bezier(arc_data)

# # 결과 출력
# print("Converted Bezier Curve Data:")
# print(bezier_curve_data)

In [ ]:
# # 일단 안씀
# # TODO: Bezier Curve로 Arc 변환하는건데 개구림
# import numpy as np

# def arc_to_bezier(arc):
#     """
#     Arc 데이터를 단일 Bezier Curve로 변환합니다.
    
#     Args:
#         arc (dict): Arc 데이터 (start_x, start_y, radius, rotation, large_arc, sweep, end_x, end_y 포함).
    
#     Returns:
#         dict: Bezier Curve의 시작점, 끝점, 제어점 2개.
#     """
#     # 시작점과 끝점 정의
#     start_x, start_y = arc["start_x"], arc["start_y"]
#     end_x, end_y = arc["end_x"], arc["end_y"]
#     radius = arc["radius"]  # 반지름
#     large_arc = arc["large_arc"]
#     sweep = arc["sweep"]

#     # 시작점과 끝점을 복소수로 변환
#     start = complex(start_x, start_y)
#     end = complex(end_x, end_y)

#     # 중심 계산
#     center = start + (end - start) / 2

#     # 시작 및 끝 각도 계산
#     theta_start = np.angle(start - center)
#     theta_end = np.angle(end - center)

#     # sweep 및 large_arc 처리
#     if not sweep:
#         theta_start, theta_end = theta_end, theta_start
#     if large_arc and abs(theta_end - theta_start) < np.pi:
#         theta_end += 2 * np.pi

#     # 각도 차이 계산
#     delta_theta = theta_end - theta_start

#     # 제어점 비율 계산
#     k = (4 / 3) * np.tan(delta_theta / 4)

#     # 시작점, 끝점 계산
#     P0_x, P0_y = start_x, start_y
#     P3_x, P3_y = end_x, end_y

#     # 제어점 계산
#     P1_x = P0_x + k * radius * np.cos(theta_start)
#     P1_y = P0_y + k * radius * np.sin(theta_start)

#     P2_x = P3_x - k * radius * np.cos(theta_end)
#     P2_y = P3_y - k * radius * np.sin(theta_end)

#     # 반환 형식
#     return {
#         "start_x": P0_x, "start_y": P0_y,
#         "control1_x": P1_x.real, "control1_y": P1_y.real, #TODO: 허수부 제거
#         "control2_x": P2_x.real, "control2_y": P2_y.real,
#         "end_x": P3_x, "end_y": P3_y,
#     }


In [ ]:
# 3차원으로 변환
import numpy as np

def euler_to_rotation_matrix(rx_deg, ry_deg, rz_deg):
    """
    Rx, Ry, Rz (degree) 오일러 각을 받아서
    X축 -> Y축 -> Z축 순서로 회전하는 3x3 회전 행렬을 반환.
    """
    # 각도를 radian으로 변환
    rx = np.deg2rad(rx_deg)
    ry = np.deg2rad(ry_deg)
    rz = np.deg2rad(rz_deg)

    # 회전 행렬 구성

    # 1) X축 회전 행렬
    Rx = np.array([
        [1,          0,           0],
        [0,  np.cos(rx), -np.sin(rx)],
        [0,  np.sin(rx),  np.cos(rx)]
    ])

    # 2) Y축 회전 행렬
    Ry = np.array([
        [ np.cos(ry), 0, np.sin(ry)],
        [          0, 1,          0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])

    # 3) Z축 회전 행렬
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz),  np.cos(rz), 0],
        [         0,           0, 1]
    ])

    # 순서: X축 회전 -> Y축 회전 -> Z축 회전 (Intrinsic)
    # 최종 회전 행렬 = Rz * Ry * Rx (행렬 곱 순서 유의)
    R = Rz @ Ry @ Rx

    return R

def transform_2d_points_to_3d(points_2d, translation, rotation):
    """
    2D 좌표 리스트(points_2d)를 rotation, translation 정보를 사용해
    3D 좌표로 변환.
    
    Parameters
    ----------
    points_2d : list of (x, y)
        변환 대상 2D 좌표들
    translation : list or tuple
        [Tx, Ty, Tz] 형태
    rotation : list or tuple
        [Rx_deg, Ry_deg, Rz_deg] 형태 (도 단위)
    
    Returns
    -------
    transformed_points_3d : np.ndarray
        변환된 3D 좌표 (N x 3)
    """
    # 1) 2D -> 3D로 확장 (z=0 가정)
    points_3d = np.array([[x, y, 0.0] for (x, y) in points_2d])

    # 2) 오일러 각 -> 회전 행렬
    Rx_deg, Ry_deg, Rz_deg = rotation
    R = euler_to_rotation_matrix(Rx_deg, Ry_deg, Rz_deg)

    # 3) 회전 적용
    # 행렬 곱: (N x 3) · (3 x 3) = (N x 3)
    points_rotated = points_3d @ R.T

    # 4) 평행이동 적용
    Tx, Ty, Tz = translation
    points_translated = points_rotated + np.array([Tx, Ty, Tz])

    return points_translated


data_dict = {
    "pattern": {
        "panels": {
            "skirt_front_0": {
                "translation": [-13.1197528, -15.0477905, 25.0],
                "rotation": [0.0, 0.0, 29.0]
            }
        }
    }
}

# 변환 대상 2D 좌표 예시
points_2d = [(0, 0), (10, 0), (0, 10), (10,10)]

# 딕셔너리에서 translation, rotation 읽어오기
panel_data = data_dict["pattern"]["panels"]["skirt_front_0"]
translation = panel_data["translation"]   # [Tx, Ty, Tz]
rotation = panel_data["rotation"]         # [Rx, Ry, Rz] (deg)

# 변환
transformed_points_3d = transform_2d_points_to_3d(points_2d, translation, rotation)

print("변환된 3D 좌표:")
print(transformed_points_3d)

In [ ]:
# 2번
# 각 panel에 대한 start, end 좌표를 저장해 놓는 함수
import math
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc
# TODO: 2024.01.19 15:29 name 추가함.
def calculate_edge_length(edge, name):
    """
    각 엣지의 길이를 계산.
    """
    if isinstance(edge, Line):
        return abs(edge.end - edge.start)
    elif isinstance(edge, QuadraticBezier):
        return edge.length()
    elif isinstance(edge, CubicBezier):
        return edge.length()
    elif isinstance(edge, Arc):
        return edge.length()
    else:
        raise ValueError(f"Unsupported edge type: {type(edge)}")

def calculate_fstart_fend(path, panel_name):
    """
    전체 엣지에 대해 fstart와 fend를 계산.
    """
    total_length = sum(calculate_edge_length(edge, name) for edge, name in zip(path, panel_name))
    fstart = 0.0
    results = []

    for idx, edge in enumerate(path):  # enumerate를 사용하여 번호 부여
        edge_length = calculate_edge_length(edge)
        fend = fstart + (edge_length / total_length)
        results.append({
            "edge_id": idx,  # 엣지 번호
            "edge": str(edge),  # Edge를 문자열로 변환하여 저장
            "fstart": round(fstart, 6),
            "fend": round(fend, 6)
        })
        fstart = fend

    return results

# 여러 패널의 정보를 저장할 딕셔너리
panel_fstart_fend_info = {}

# 패널 리스트 (예: pattern.panel_order())
panel_list = pattern.panel_order()

# 각 패널에 대해 fstart, fend 계산
for panel_name in panel_list:
    path = panel_svg_path_dict[panel_name][0]  # 패널에 해당하는 경로 정보 가져오기
    fstart_fend_results = calculate_fstart_fend(path, panel_name) # TODO: 2024.01.19 14:33 panel_name 넣기
    panel_fstart_fend_info[panel_name] = fstart_fend_results  # 결과를 딕셔너리에 저장

# panel_fstart_fend_info에 패널 정보, edge, start, end, edge_id 까지 저장 되어있다.
# 결과 출력 (딕셔너리 형태)
import pprint
pprint.pprint(panel_fstart_fend_info)

In [2]:
# 3번
### 원래꺼 
# 이게 2024.01.14 12:58 + (15:55 기준 최신본.
# TODO: Pattern List + (InstanceDataList)에 대한 정보를 json으로 넣는 코드.
# ++ Bezier Curve, Straight가 아닌
# Arc , Qudratic Curve, Cubic Curve에 대해서는 어떻게 다룰지. 
# InstanceDataList 생성

# InstanceDataList 추가 루프

import pandas as pd
import json
import math

# 패널 데이터프레임 생성
panel_dataframes = [process_path_data(drawn_pattern) for drawn_pattern in drawn_pattern_list]
panel_names = list(pattern.panel_order())

# Initialize the top-level JSON structure
garment_data = {
    "FabricList": [],
    "GradingRuleTableList": [],
    "PatternList": [],
    "SymmetricDataList": [],
    "InstanceDataList": [],
    "SeamLinePairGroupList": [],
}

# Process each panel
for panel_name, df in zip(panel_names, panel_dataframes):
    # Initialize JSON structure for the panel
    panel_data = {
        "Name": panel_name,
        "fGrainlineAngle": 0.0,
        "ID": panel_name,
        "strSuperImposeSide": "None",
        "CurrentFabricUUID": "DWX6ey",
        "IsClosed": False,
        "InternalLineList": [],
        "ButtonHeadList": [],
        "ButtonHoleList": [],
        "AnnotationList": [],
        "NotchList": [],
        "IsHalfSymmetric": False,
        "ShapeInfo": {
            "IsSlashed": False,
            "LineList": [],
            "HalfSymmetryPointIDMap": {},
            "HalfSymmetryLineIDMap": {}
        },
        "ArrangementPointDataMap": {
                "PointName": "Body_Front_Center_3",
                "fOffSetX": 0.0,
                "fOffSetY": 0.05,
                "fAngle": 180.0
        }
    }
    # InstanceDataList에 json에 추가
    instance_data = {
        "OriginPatternID": panel_name,  # 패널 이름을 OriginPatternID로 사용
        "InstancePatternIDArray": []   # 빈 리스트로 추가
    }
    garment_data["InstanceDataList"].append(instance_data)
    # 패널별 좌표 ID 초기화
    coordinate_id_map = {}
    point_counter = 0  # 패널 내 좌표 카운터

    # Generate LineList and PointList for the panel
    for edge_idx, row in df.iterrows():
        line_id = f"{panel_name}_edge_{edge_idx}"
        point_list = []
        columns_x = [col for col in row.index if col.endswith('_x')]
        columns_y = [col for col in row.index if col.endswith('_y')]
        
        if row["type"] == "Bezier Curve":
            row_type = ["Straight", "Bezier Curve", "Bezier Curve", "Straight"]
            idx = 0
            for col_x, col_y in zip(columns_x, columns_y):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id, 
                        "PointType": row_type[idx],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
                    idx +=1 
        # TODO: Arc 타입에 대해 베지어로 변환하여 Json에 넣는 작업.
        elif row["type"] == "Arc":
            arc_bezier = arc_to_bezier(row)
            columns_x = [key for key in row.keys() if key.endswith('_x')]
            columns_y = [key for key in row.keys() if key.endswith('_y')]
            # 사용하지 않을 키 필터링
            excluded_keys = {"controla_x", "controlb_x", "controla_y", "controlb_y"}
            columns_x = [col for col in columns_x if col not in excluded_keys]
            columns_y = [col for col in columns_y if col not in excluded_keys]
            print(columns_x, columns_y)
            
            row_type = ["Straight", "Bezier Curve", "Bezier Curve", "Straight"]
            idx = 0
            for col_x, col_y in zip(columns_x, columns_y):  # Skip 'type' and go by pairs
                print(arc_bezier[col_x], arc_bezier[col_y])
                x, y = arc_bezier[col_x], arc_bezier[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id, 
                        "PointType": row_type[idx],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
                    idx +=1          
        elif row["type"] == "Straight":
            # Iterate through columns 2 by 2 for x and y
            for col_x, col_y in zip(columns_x, columns_y):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id,
                        "PointType": row["type"],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
    

        # Add Line to LineList
        line = {"ID": line_id, "PointList": point_list}
        panel_data["ShapeInfo"]["LineList"].append(line)

    # Add the panel data to the PatternList
    garment_data["PatternList"].append(panel_data)

# Convert to JSON string with indentation
json_output = json.dumps(garment_data, indent=4)

# Save to a file (optional)
output_file = "multiple_panels_data3.json"
with open(output_file, "w") as file:
    file.write(json_output)

# 결과 출력
print(json_output)

NameError: name 'drawn_pattern_list' is not defined

In [ ]:
# 4번
# 2024.01.14 14:33 기준 2:51 기준
# 2024.01.14 TODO: Seam Line Pair Group json으로 넣는 코드
# 2024.01.16 TODO: front , back 에 따라 Pair List의 Direction 결정 넣기
# SeamLinePairGroupList 생성
####TODO: 여기부터 내가 임의로 테스트 해봄 2024.01.19 16:16
file_path = '/media/hjp/db6095ca-a560-4c3a-90ad-b667ec189671/REFERENCES/3D_VTO/GarmentCode/GarmentCode/ANALYSIS_josef/nique_panel_names.csv'
data = pd.read_csv(file_path)

# 열 이름 설정
data.columns = ['name']

# 분류 로직 추가
data['classification'] = data['name'].apply(
    lambda x: 'back' if '_back' in x or '_b' in x or '_btorso' in x else
            'front' if '_front' in x or '_f' in x or '_ftorso' in x else
            'other'
)

# 각 분류에 따른 리스트 생성
front_list = data[data['classification'] == 'front']['name'].tolist()
back_list = data[data['classification'] == 'back']['name'].tolist()
other_list = data[data['classification'] == 'other']['name'].tolist()    
#### TODO: 여기까지    

garment_data["SeamLinePairGroupList"] = []
fabric_group = {
    "FabricName" : "Default Fabric",
    "FabricType" : "None",
    "FabricContent" : "None",
    "strBaseColorHexCode" : "#FFFFFF",
    "FabricUUID" : "DWX6ey"
    }
# SeamLinePairGroupList 생성 루프
for idx, value in enumerate(stitch_dict.values()):
    

    
    # "Name" 생성
    group_name = f"SeamLineGroup_{idx}"

    # value에서 첫 번째와 두 번째 ShapeID 정보 추출
    first_info = value[0]
    second_info = value[1]
        
    # 첫 번째 정보
    first_panel = first_info["panel"]
    first_edge = first_info["edge"]
    first_shape_id = f"{first_panel}"

    # 첫 번째 fStart와 fEnd 추출
    first_fstart_fend = next(
        (item for item in panel_fstart_fend_info[first_panel] if item["edge_id"] == first_edge),
        None
    )
    first_fstart = first_fstart_fend["fstart"]
    first_fend = first_fstart_fend["fend"]
    first_point= first_fstart_fend["edge"]

    if first_panel in front_list:
        first_direction = select_direction_front(first_fstart_fend)
    elif first_panel in back_list:
        first_direction = select_direction_back(first_fstart_fend)
    else:
        first_direction = False
    
    print(first_direction)
    if first_direction == False:
        first_fstart, first_fend = first_fstart_fend["fend"], first_fstart_fend["fstart"]
        
    # 두 번째 정보
    second_panel = second_info["panel"]
    second_edge = second_info["edge"]
    second_shape_id = f"{second_panel}" #TODO: 이부분 제외 '_edge_{second_edge}"'

    # 두 번째 fStart와 fEnd 추출
    second_fstart_fend = next(
        (item for item in panel_fstart_fend_info[second_panel] if item["edge_id"] == second_edge),
        None
    )
    second_fstart = second_fstart_fend["fstart"]
    second_fend = second_fstart_fend["fend"]
    second_point = second_fstart_fend["edge"]
    # TODO 2024.01.19 14:30 direeciton 넣기 
    
    if second_panel in front_list:
        second_direction = select_direction_front(second_fstart_fend)
    elif second_panel in back_list:
        second_direction = select_direction_back(second_fstart_fend)
    else:
        second_direction = False

    if second_direction == False:
        second_fstart, second_fend = second_fstart_fend["fend"], second_fstart_fend["fstart"]
        
    # JSON 구조 생성
    seam_line_pair_group = {
        "Name": group_name,
        "bIsTurned": False,
        "PairList": [
            {
                "First": {
                    "ShapeID": first_shape_id,
                    "LengthParam": {
                        "fStart": first_fstart,
                        "fEnd": first_fend
                    },
                    "Direction": first_direction
                },
                "Second": {
                    "ShapeID": second_shape_id,
                    "LengthParam": {
                        "fStart": second_fstart,
                        "fEnd": second_fend
                    },
                    "Direction": second_direction # TODO True로 일단 default
                }
            }
        ],
        "FoldData": {
            "iAngle": 180,
            "iStrength": 5
        }
    }

    # SeamLinePairGroupList에 추가
    garment_data["SeamLinePairGroupList"].append(seam_line_pair_group)

garment_data["FabricList"].append(fabric_group)    

# JSON 출력
json_output = json.dumps(garment_data, indent=4)

# Save to file (optional)
output_file = "garment_with_seam_lines.json"
with open(output_file, "w") as file:
    file.write(json_output)

# Print the result
print(json_output)